In [ ]:
#from transformers import pipeline

#summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

#ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
#A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
#Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
#In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
#Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
#2010 marriage license application, according to court documents.
#Prosecutors said the marriages were part of an immigration scam.
#On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
#After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
#Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
#All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
#Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
#Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
#The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
#Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
#Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
#If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
#"""
#print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))
#[{'summary_text': 'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.'}]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[{'summary_text': 'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.'}]


In [ ]:
from datasets import load_dataset, Dataset
from deep_translator import GoogleTranslator
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Create a directory in Google Drive to save the batches
os.makedirs('/content/drive/MyDrive/google_translated_batches', exist_ok=True)

# List already saved batches
saved_batches = os.listdir('/content/drive/MyDrive/google_translated_batches')
saved_indices = [int(batch.split('_')[1]) for batch in saved_batches if batch.startswith('batch_')]
from datasets import load_from_disk

# Load one of the saved batches
loaded_batch = load_from_disk("/content/drive/MyDrive/google_translated_batches/batch_0")

# Check the content of the loaded batch
print(loaded_batch[0])  # Print the first example in the batch


# Back-translation function using deep-translator
def back_translate(text, src_lang="en", tgt_lang="es"):
    translated_text = GoogleTranslator(source=src_lang, target=tgt_lang).translate(text)
    back_translated_text = GoogleTranslator(source=tgt_lang, target=src_lang).translate(translated_text)
    return back_translated_text

# Function to apply back-translation on the dataset
def back_translate_dataset(example):
    example['back_translated_summary'] = back_translate(example['summary'])
    return example

# Load the entire dataset
ds6 = load_dataset("glnmario/news-qa-summarization")

# Define the batch size for periodic saving
batch_size = 100

# Process and save in batches
for i in range(0, len(ds6['train']), batch_size):
    # Skip batches that are already processed
    if i in saved_indices:
        print(f"Skipping batch {i} to {i + batch_size} (already processed)")
        continue

    # Process the current batch
    batch = ds6['train'].select(range(i, min(i + batch_size, len(ds6['train']))))
    augmented_batch = batch.map(back_translate_dataset)

    # Convert the processed batch to dictionary format for saving
    output_ds = Dataset.from_dict(augmented_batch.to_dict())

    # Save each batch separately in Google Drive
    output_ds.save_to_disk(f"/content/drive/MyDrive/google_translated_batches/batch_{i}")
    print(f"Batch {i} to {i + batch_size} saved successfully to Google Drive.")

# After saving, list the saved directories in Google Drive
print("Batches saved successfully in Google Drive:")
print(os.listdir("/content/drive/MyDrive/google_translated_batches"))






Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/715 [00:00<?, ?B/s]

data.jsonl:   0%|          | 0.00/40.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10388 [00:00<?, ? examples/s]

...

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

Batch 10200 to 10300 saved successfully to Google Drive.


Map:   0%|          | 0/88 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/88 [00:00<?, ? examples/s]

Batch 10300 to 10400 saved successfully to Google Drive.
Batches saved successfully in Google Drive:
['batch_0', 'batch_100', 'batch_200', 'batch_300', 'batch_400', 'batch_500', 'batch_600', 'batch_700', 'batch_800', 'batch_900', 'batch_1000', 'batch_1100', 'batch_1200', 'batch_1300', 'batch_1400', 'batch_1500', 'batch_1600', 'batch_1700', 'batch_1800', 'batch_1900', 'batch_2000', 'batch_2100', 'batch_2200', 'batch_2300', 'batch_2400', 'batch_2500', 'batch_2600', 'batch_2700', 'batch_2800', 'batch_2900', 'batch_3000', 'batch_3100', 'batch_3200', 'batch_3300', 'batch_3400', 'batch_3500', 'batch_3600', 'batch_3700', 'batch_3800', 'batch_3900', 'batch_4000', 'batch_4100', 'batch_4200', 'batch_4300', 'batch_4400', 'batch_4500', 'batch_4600', 'batch_4700', 'batch_4800', 'batch_4900', 'batch_5000', 'batch_5100', 'batch_5200', 'batch_5300', 'batch_5400', 'batch_5500', 'batch_5600', 'batch_5700', 'batch_5800', 'batch_5900', 'batch_6000', 'batch_6100', 'batch_6200', 'batch_6300', 'batch_6400', 

In [ ]:
from datasets import load_from_disk


from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')
# Load one of the saved batches
loaded_batch = load_from_disk("/content/drive/MyDrive/google_translated_batches/batch_0")

# Check the content of the loaded batch
for idx in range(5):  # Print the first 5 examples to inspect
    print(f"Original Summary: {loaded_batch['summary'][idx]}")
    print(f"Back-Translated Summary: {loaded_batch['back_translated_summary'][idx]}")
    print('-' * 80)


Mounted at /content/drive
Original Summary: British soldiers train in mock Afghan village before deployment to Afghanistan .
Village features Afghan asylum-seekers as vendors, elders, Afghan forces .
Taliban militants are played by Nepalese Ghurkha soldiers .
Village trains soldiers to understand Afghan customs, respect Afghan culture .
Back-Translated Summary: British soldiers train in a mock Afghan village before being sent to Afghanistan.
The village features Afghan asylum seekers as vendors, elders and Afghan forces.
Taliban militants are played by Nepalese Ghurkha soldiers.
The village trains soldiers to understand Afghan customs and respect Afghan culture.
--------------------------------------------------------------------------------
Original Summary: Latest technologies debut at the 2009 Detroit Auto Show .
Chrysler, Mercedes-Benz, Toyota and MINI show what electric cars of future will be .
Toyota confirms plans to add as many as 10 new gas/electric hybrid vehicles .
The "char

In [ ]:
import nltk
from nltk.corpus import wordnet
import nlpaug.augmenter.word as naw
from datasets import load_dataset
from datasets import load_from_disk
from google.colab import drive
import os

nltk.download('wordnet')

# Mount Google Drive
drive.mount('/content/drive')
# Load the saved batch
loaded_batch = load_from_disk("/content/drive/MyDrive/google_translated_batches/batch_0")

# Display the first example to verify content
print(loaded_batch[0])

ds6 = load_dataset("glnmario/news-qa-summarization")
# Select the first 5 examples
first_five = ds6['train'].select(range(5))

# Apply WordNet synonym replacement
def synonym_replace(text):
    words = text.split()
    new_words = []
    for word in words:
        synonyms = wordnet.synsets(word)
        if synonyms:
            synonym = synonyms[0].lemmas()[0].name()  # Use the first lemma
            new_words.append(synonym)
        else:
            new_words.append(word)
    return " ".join(new_words)

# Apply NLPAug synonym augmentation
synonym_aug = naw.SynonymAug(aug_src='wordnet')

def augment_text(text):
    return synonym_aug.augment(text)

# Process the first 5 samples
processed_first_five = first_five.map(lambda x: {
    'summary': x['summary'],
    'synonym_replaced_summary': synonym_replace(x['summary']),
    'augmented_summary': augment_text(x['summary'])
})

# Display results
# Display results with improved readability, keeping full sentences
for idx in range(5):
    print(f"Original Summary:\n{processed_first_five['summary'][idx]}")
    print("\nSynonym-Replaced Summary:")
    print("\n".join(processed_first_five['synonym_replaced_summary'][idx].split('. ')))  # Keep all sentences with line breaks

    print("\nNLPAug Augmented Summary:")
    # Convert NLPAug output to string if it's a list, then add line breaks
    nlpaug_output = processed_first_five['augmented_summary'][idx]
    if isinstance(nlpaug_output, list):
        nlpaug_output = " ".join(nlpaug_output)
    print("\n".join(nlpaug_output.split('. ')))  # Keep all sentences with line breaks

    print("-" * 80)






[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
{'story': '\'SINDH KALAY\', England (CNN) -- The aroma of freshly baking flatbread wafts through the air as a unit of British soldiers position themselves for a quick patrol around the village of Sindh Kalay. A British soldier on patrol in the mock Afghan village of Sindh Kalay. Market vendors hawk grapes and melons, as a group of village elders sit smoking water pipes and suspicious-looking men lurk beside battered motorcycles. What should the soldiers do? Conduct a weapons search? Approach the village elders first? In the complex political and cultural terrain of Afghanistan, what is the best course of action? Except this is not Afghanistan. It\'s Norfolk, England. Instead of the Hindu Kush mountains, it is the green ladscape and tidy farmhouses of the English countryside that stretch out behind them. Welcome to the British Army\'s state-of-the art training

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Original Summary:
British soldiers train in mock Afghan village before deployment to Afghanistan .
Village features Afghan asylum-seekers as vendors, elders, Afghan forces .
Taliban militants are played by Nepalese Ghurkha soldiers .
Village trains soldiers to understand Afghan customs, respect Afghan culture .

Synonym-Replaced Summary:
British soldier train inch mock afghan village earlier deployment to Afghanistan 
village feature afghan asylum-seekers arsenic vendors, elders, afghan force 
Taliban militant are play by Nepalese Ghurkha soldier 
village train soldier to understand afghan customs, respect afghan culture .

NLPAug Augmented Summary:
British soldiers train in mock Afghan village before deployment to Islamic state of afghanistan
Settlement features Afghan asylum - seekers as vendors, elders, Afghan hound forces
Taliban militant are played by Nepalese Ghurkha soldiers
Village train soldiers to infer Afghanistani customs, respect Afghan culture.
---------------------------

In [ ]:
!pip install deep-translator


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.9 MB/s eta 0:00:00


In [ ]:
!pip install deep-translator


In [ ]:
!pip install nlpaug



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 7.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_from_disk

# Load the saved batch (replace `back_translated_batch_0` with your batch path)
loaded_batch = load_from_disk("back_translated_batch_0")
# Print the first sample in the loaded batch
print(loaded_batch[0])

# Print multiple samples (optional)
for i in range(5):
    print(f"Sample {i}: {loaded_batch[i]}")


FileNotFoundError: Directory back_translated_batch_0 not found

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00


In [ ]:
pip install datasets googletrans==4.0.0-rc1


  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 993.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [ ]:
!python -c "from datasets import load_dataset; from googletrans import Translator; print('All required libraries are installed.')"

All required libraries are installed.


In [ ]:
from datasets import load_dataset
from googletrans import Translator
import time

# Load the CNN/DailyMail dataset version 3.0.0 (train split)
ds = load_dataset("abisee/cnn_dailymail", "3.0.0", split='train')

# Initialize Google Translator
translator = Translator()

# Back-translation using Google Translate API: English -> Spanish -> English
def back_translate(text, src_lang='en', tgt_lang='es'):
    try:
        # Translate from English to Spanish and back to English
        translated = translator.translate(text, src=src_lang, dest=tgt_lang).text
        back_translated = translator.translate(translated, src=tgt_lang, dest=src_lang).text
        return back_translated
    except Exception as e:
        return text  # Fallback to original text if translation fails

# Apply back-translation to each example in the dataset
def augment_with_back_translation(example):
    try:
        example['augmented_article'] = back_translate(example['article'])
    except Exception:
        example['augmented_article'] = example['article']  # If translation fails, use original article
    return example

# Start back-translation and measure the time
start_time = time.time()
ds_augmented = ds.map(augment_with_back_translation)
end_time = time.time()

# Print the time taken
print(f"Back-translation completed in {end_time - start_time} seconds")

# Save the augmented dataset to disk
ds_augmented.save_to_disk("path/to/save/augmented_dataset")

# Optional: Print the original and augmented article for the first example
print("Example 1 - Augmented Article:")
print(ds_augmented[0]['augmented_article'])